In [ ]:
"""
This Jupyter Notebook is used to test the SVMBDT implementation for the Mjosa dataset. 
It also test different other ML algortithms from the function SingleMachineLearningTest(). 
The labeled figures are store on the folder /Results/Mjøsa_Results and the results are printed in the console. 
"""

In [1]:
#Imports 
import pip
import glob, os
from sklearn import svm 
import scipy.io as sc
import numpy as np
from HelperFunctions_ import write_dataset_to_file, split_dataset_paths, subFrame,preprocessing_data, newColorLabels
from HelperFunctions_ import  combinePictures, classesInLabels, combineLabelClasses, lessBands, reshape_sj, SingleMachineLearningTest
from SVMBDT import SvmDesionTreeTrain, SvmDesionTreePredict, RetrieveSubData, CombineMultiBranch, MeanAccuracy, SupportVectorsSvmBranches, TotalAndMeanSupportVectors
from spectral import *
import pylab
import tables as tab
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import LinearSVC, SVC, OneClassSVM
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.ensemble import StackingClassifier
import copy
import tables as ta
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.patches as mpatches
from sklearn.ensemble import RandomForestClassifier


In [2]:
#Finding path names 
Mjosa_data = split_dataset_paths("../Datasets/2022_07_11_Mjosa/2022_07_11_Mjosa/", ["he5", "csv", "jpg"])

In [3]:
#Amount of dataset should be handled
N_datasets = 4

#Open files 
open_files_array = []
for i in range(N_datasets):
    file = tab.open_file(Mjosa_data["he5"][i])
    open_files_array.append(file.get_node("/HDFEOS/SWATHS/PRS_L1_HCO/")['Data Fields']['VNIR_Cube'][:,5:,:])

#Load the labels 
load_labels_array = []
for i in range(N_datasets):
    file = Mjosa_data["csv"][i]
    load_labels_array.append(np.loadtxt(file, dtype=np.ubyte))

In [4]:
data = open_files_array[1]
labels = load_labels_array[1]

In [5]:
print(len(data[0]))

61


In [6]:
f1 = ta.open_file("../Datasets/2022_07_11_Mjosa/2022_07_11_Mjosa/PRS_L1_STD_OFFL_20220711104640_20220711104644_0001.he5")

d3 = f1.get_node("/HDFEOS/SWATHS/PRS_L1_HRC/")

mjosa_new_cube = np.zeros((1000,1000,3))
mjosa_new_cube[:,:,0] = d3['Data Fields']['VNIR_Cube'][:,31,:]
mjosa_new_cube[:,:,1] = d3['Data Fields']['VNIR_Cube'][:,43,:]
mjosa_new_cube[:,:,2] = d3['Data Fields']['VNIR_Cube'][:,56,:]
mjosa_new_cube /= mjosa_new_cube.max()/2
mjosa_new_cube[mjosa_new_cube > 1 ] = 1


plt.imshow(mjosa_new_cube)
plt.axis('off')
plt.savefig("../Results/Mjøsa_Results/Mjøsa_Data", dpi = 300, bbox_inches="tight")
plt.close()

In [7]:
X,y = preprocessing_data(data,labels,61)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.95, random_state=1)

In [8]:
mjosa_color_label = {0: [255,165,0], 1: [34,139,34], 2: [53, 97, 4], 3: [5, 216, 105], 4 : [0,0,150], 5 : [210,210,210]}
t=255
cmap = {}

for i in range(len(mjosa_color_label)): 
    cmap[i] = np.array(mjosa_color_label[i] + [255])/255


# 1) Spruce, 2) Pine , 3) Decidous (leaf), 4) Water, 5) Clouds, and 0) other
mjosa_classes = {0: "Other", 1: "Spruce",2: "Pine", 3:"Decidous", 4 : "Water", 5 : "Clouds"}

shape = int(np.sqrt(len(y)))

yout_color = newColorLabels(y,shape,shape,mjosa_color_label)

plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True

patches = [mpatches.Patch(color=cmap[i], label=mjosa_classes[i]) for i in mjosa_color_label] 

plt.imshow(yout_color)
plt.axis('off')
#plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0. )
plt.savefig("../Results/Mjøsa_Results/Mjøsa_GT", dpi = 300, bbox_inches="tight")
plt.close()

C:\Users\jonas\AppData\Local\Temp\ipykernel_35668\82161751.py:24: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.savefig("../Results/Mjøsa_Results/Mjøsa_GT", dpi = 300, bbox_inches="tight")


In [15]:
figLegend = pylab.figure(figsize = (1.3,1.4))
figLegend.legend(handles=patches,loc = 'upper left')
figLegend.savefig("../Results/Mjøsa_Results/Mjøsa_Classes", dpi=500)

<Figure size 130x140 with 0 Axes>

In [12]:
# 1) Spruce, 2) Pine , 3) Decidous (leaf), 4) Water, 5) Clouds, and 0) other
tb = {0 : [[4],[0,1,2,3,5]], 1 : [[],[[5],[0,1,2,3]]], 2 : [[],[[0],[1,2,3]]], 3 : [[],[[1],[2,3]]], 4 : [[],[[2],[3]]]}

iter = 1e5

sbm = {0 : LinearSVC(class_weight= "balanced", max_iter=iter),
       1 : [False, LinearSVC(class_weight= "balanced", max_iter=iter)], 
       2 : [False, LinearSVC(class_weight= "balanced", max_iter=iter)],
       3 : [False, LinearSVC(class_weight= "balanced", max_iter=iter)], 
       4 : [False, LinearSVC(class_weight= "balanced", max_iter=iter)]}


svm_tree_branches = SvmDesionTreeTrain(X_train, y_train, tb, sbm)

The training time is: 130.383 sec.


In [13]:
predicted_label = SvmDesionTreePredict(X, svm_tree_branches, tb)

The first branch: 0.101
The prediction time is: 4.663 sec.


In [14]:
sum(predicted_label == y)/len(y)

0.725833

In [52]:
linear_ovo_yout, linear_ovr_yout, rbf_yout, kmeans_yout, knearest_yout,random_forest_yout = SingleMachineLearningTest(X,y,X_train, y_train,1e6,5)

Linear 1-vs-1 Training Time: 238.309sec.
Linear 1-vs-Rest Training Time: 2688.079sec.
RBF Training Time: 214.895499sec.
Kmeans Training Time: 0.751sec.
KNearestNeighbor Training Time: 0.014sec.
KNearestNeighbor Training Time: 28.862sec.
Linear 1-vs-1 Predict Time: 1047.757sec.
Linear 1-vs-Rest Predict Time: 0.252sec.
RBF Predict Time: 3474.887sec.
KMeans Predict Time: 0.213sec.
KNearestNeighbor Predict Time: 47.078sec.
Random Forest Predict Time: 14.657sec.
Linear 1-vs-1 Accuracy: 68.46%
Linear 1-vs-Rest Accuracy: 73.35000000000001%
RBF Accuracy: 68.39%
KMeans Accuracy: 8.48%
KNearestNeighbour Accuracy: 75.2%
Random Forest Accuracy: 78.7%
Linear 1-vs-1 Total SVM Support_Vectors : 30245
RBF Total SVM Support_Vectors : 31133
Linear 1-vs-1 Average SVM Support_Vectors : 5040.833
RBF Average SVM Support_Vectors : 5188.833


In [54]:
yout_color = newColorLabels(linear_ovo_yout,shape, mjosa_color_label)

plt.imshow(yout_color)
plt.axis('off')
plt.savefig("../Results/Mjøsa_Results/Mjøsa_linear_ovo",dpi = 300, bbox_inches="tight")
plt.close()

C:\Users\jonas\AppData\Local\Temp\ipykernel_23008\2418608748.py:5: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.savefig("../Results/Mjøsa_Results/Mjøsa_linear_ovo",dpi = 300, bbox_inches="tight")


In [55]:
yout_color = newColorLabels(linear_ovr_yout,shape, mjosa_color_label)

plt.imshow(yout_color)
plt.axis('off')
plt.savefig("../Results/Mjøsa_Results/Mjøsa_linear_ovr",dpi = 300, bbox_inches="tight")
plt.close()

C:\Users\jonas\AppData\Local\Temp\ipykernel_23008\778744977.py:5: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.savefig("../Results/Mjøsa_Results/Mjøsa_linear_ovr",dpi = 300, bbox_inches="tight")


In [56]:
yout_color = newColorLabels(rbf_yout,shape, mjosa_color_label)

plt.imshow(yout_color)
plt.axis('off')
plt.savefig("../Results/Mjøsa_Results/Mjøsa_RBF",dpi = 300, bbox_inches="tight")
plt.close()

C:\Users\jonas\AppData\Local\Temp\ipykernel_23008\2460380850.py:5: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.savefig("../Results/Mjøsa_Results/Mjøsa_RBF",dpi = 300, bbox_inches="tight")


In [57]:
yout_color = newColorLabels(knearest_yout,shape, mjosa_color_label)

plt.imshow(yout_color)
plt.axis('off')
plt.savefig("../Results/Mjøsa_Results/Mjøsa_KNN",dpi = 300, bbox_inches="tight")
plt.close()

C:\Users\jonas\AppData\Local\Temp\ipykernel_23008\2618902138.py:5: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.savefig("../Results/Mjøsa_Results/Mjøsa_KNN",dpi = 300, bbox_inches="tight")


In [58]:
yout_color = newColorLabels(random_forest_yout,shape, mjosa_color_label)

plt.imshow(yout_color)
plt.axis('off')
plt.savefig("../Results/Mjøsa_Results/Mjøsa_RF",dpi = 300, bbox_inches="tight")
plt.close()

C:\Users\jonas\AppData\Local\Temp\ipykernel_23008\3393792025.py:5: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.savefig("../Results/Mjøsa_Results/Mjøsa_RF",dpi = 300, bbox_inches="tight")


In [71]:
mjosa_color_label = {3: [255,165,0], 2: [34,139,34], 0: [53, 97, 4], 5: [5, 216, 105], 1 : [0,0,150], 4 : [210,210,210]}
yout_color = newColorLabels(kmeans_yout,shape, mjosa_color_label)

plt.imshow(yout_color)
plt.axis('off')
plt.savefig("../Results/Mjøsa_Results/Mjøsa_KM",dpi = 300, bbox_inches="tight")
plt.close()

C:\Users\jonas\AppData\Local\Temp\ipykernel_23008\4090974047.py:6: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.savefig("../Results/Mjøsa_Results/Mjøsa_KM",dpi = 300, bbox_inches="tight")
